In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn import tree
from sklearn.ensemble import *

In [3]:
data = pd.read_csv("/Users/dariashavronskaia/Desktop/data science/data/AirPass.csv")

In [10]:
data.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [5]:
data.isna().sum().sum()

310

In [7]:
data['Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].fillna(data['Arrival Delay in Minutes'].median())
data['Arrival Delay in Minutes'].mean()

15.133392362180475

In [20]:
data.groupby('Gender')['satisfaction'].value_counts()*100/data.shape[0]

Gender  satisfaction           
Female  neutral or dissatisfied    29.058554
        satisfied                  21.687327
Male    neutral or dissatisfied    27.608177
        satisfied                  21.645942
Name: count, dtype: float64

In [21]:
data.groupby('Type of Travel')['satisfaction'].value_counts()*100/data.shape[0]

Type of Travel   satisfaction           
Business travel  satisfied                  40.177472
                 neutral or dissatisfied    28.785225
Personal Travel  neutral or dissatisfied    27.881506
                 satisfied                   3.155798
Name: count, dtype: float64

In [22]:
data.groupby('Class')['satisfaction'].value_counts()*100/data.shape[0]

Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: count, dtype: float64

In [23]:
data['satisfaction'] = data['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
data['Customer Type'] = data['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
data['Type of Travel'] = data['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
data['Gender'] = data['Gender'].map({'Male': 0, 'Female': 1})

In [27]:
data = pd.get_dummies(data)

In [28]:
data.shape

(103904, 27)

In [38]:
X = data.drop('satisfaction', axis = 1)
y = data['satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=26)

In [39]:
y_test.shape

(20781,)

In [45]:
scaler = StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 

In [47]:
X_test[0][0]

1.193295224074621

In [54]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
pred_test = lr.predict(X_test)
f1_score(pred_test, y_test)

0.8550407188991856

In [57]:
dt = tree.DecisionTreeClassifier(random_state=26)
model_ada = AdaBoostClassifier(dt, random_state=26, learning_rate=0.01)
model_ada.fit(X_train, y_train)
pred_test = model_ada.predict(X_test)
f1_score(pred_test, y_test)

0.9366046435040343

In [62]:
model_for_gs = GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
grid_search_cv = GridSearchCV(model_for_gs, param_grid=params, scoring=make_scorer(f1_score), cv=3, verbose=3)
grid_search_cv.fit(X_train, y_train)
grid_search_cv.best_params_

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.873 total time=   0.1s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.870 total time=   0.1s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.871 total time=   0.1s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.880 total time=   0.2s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.878 total time=   0.2s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.875 total time=   0.2s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.901 total time=   0.4s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.896 total time=   0.4s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.897 total time=   0.4s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   0.7s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   0.7s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

{'learning_rate': 1.0, 'n_estimators': 128}

In [64]:
grid_search_cv.best_score_

0.9483851654904

In [65]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(random_state=26)
model_xgb.fit(X_train, y_train)
pred_test = model_xgb.predict(X_test)
f1_score(pred_test, y_test)

0.9574894279991099

In [66]:
from catboost import CatBoostClassifier
model_cb = CatBoostClassifier(random_state=26)
model_cb.fit(X_train, y_train)
pred_test = model_cb.predict(X_test)
f1_score(pred_test, y_test)

Learning rate set to 0.068023
0:	learn: 0.6018180	total: 66ms	remaining: 1m 5s
1:	learn: 0.5117873	total: 73.1ms	remaining: 36.5s
2:	learn: 0.4550670	total: 78.7ms	remaining: 26.2s
3:	learn: 0.4109980	total: 84.5ms	remaining: 21s
4:	learn: 0.3586407	total: 90.5ms	remaining: 18s
5:	learn: 0.3283923	total: 96.2ms	remaining: 15.9s
6:	learn: 0.3068418	total: 102ms	remaining: 14.5s
7:	learn: 0.2872784	total: 108ms	remaining: 13.4s
8:	learn: 0.2702372	total: 114ms	remaining: 12.5s
9:	learn: 0.2568042	total: 120ms	remaining: 11.8s
10:	learn: 0.2460247	total: 125ms	remaining: 11.2s
11:	learn: 0.2371698	total: 131ms	remaining: 10.8s
12:	learn: 0.2227433	total: 137ms	remaining: 10.4s
13:	learn: 0.2113301	total: 142ms	remaining: 10s
14:	learn: 0.2019337	total: 148ms	remaining: 9.72s
15:	learn: 0.1969025	total: 154ms	remaining: 9.46s
16:	learn: 0.1919289	total: 160ms	remaining: 9.23s
17:	learn: 0.1882536	total: 165ms	remaining: 9s
18:	learn: 0.1816900	total: 170ms	remaining: 8.8s
19:	learn: 0.1781

0.9600177945837735

In [70]:
from catboost.utils import get_confusion_matrix
cm = get_confusion_matrix(model_cb, Pool(X_train, y_train))

NameError: name 'Pool' is not defined

In [72]:
pd.DataFrame(
    {
        "feature_importance": model_cb.get_feature_importance(),
        "feature_names": data.drop(columns="satisfaction").columns,
    }
).sort_values(by=["feature_importance"], ascending=False)

,feature_importance,feature_names
7,25.375656,Inflight wifi service
5,19.916849,Type of Travel
12,7.015148,Online boarding
3,6.778277,Customer Type
23,5.095334,Class_Business
10,3.826981,Gate location
18,3.503695,Checkin service
17,3.412145,Baggage handling
4,3.209745,Age
14,3.021277,Inflight entertainment
